In [ ]:
class Datahandling():
    def __init__(self,name_dataframe):
        self.name_dataframe = name_dataframe
        self.columns = ['iteration','wrapper','optimal_number_of_features','score_optimal_features','scoring_type',
                                            'cross_val_times','datapoints','params','running_time']
        try:
            self.current_df = pd.read_csv('results/'+name_dataframe)
            self.highest_iteration = max(self.current_df['iteration'].values)
        except Exception as e:
            #print(f"Error {e} occured")
            print('New dataframe will be made')
            if not settings["allow_creating_new_results_df"]:
                if not input("Type YES if this is oke") == "YES": 
                    raise TypeError("User decided to cancel process, so canceling. Program will crash bc no highest iter and df")
                else: #not pretty to have 2 times same else, but very tired at the moment
                    self.current_df = pd.DataFrame(columns=self.columns)
                    self.highest_iteration = 0
            
            else:
                self.current_df = pd.DataFrame(columns=self.columns)
                self.highest_iteration = 0
            
    def save_after_iter(self,final_model):
        final_model.to_csv('results/'+self.name_dataframe,index=False)
        
        
        
            
        

In [ ]:
def calculate_ranking(rfecv,final_columns,type_model):
    ranking_all_features= {}
    for i,col in enumerate([final_columns[i] for i in range(len(rfecv.support_)) if rfecv.support_[i] == True]):
        if type_model == 'svm':
            for rank, score in enumerate(sorted(rfecv.estimator_.coef_,reverse=True)):
                if rfecv.estimator_.coef_[i] == score:
                    ranking_all_features[col] = rank+1
        elif type_model == 'rf':
            print(col)
            for rank, score in enumerate(sorted(rfecv.estimator_.feature_importances_,reverse=True)):
                if rfecv.estimator_.feature_importances_[i] == score:
                    ranking_all_features[col] = rank+1
                    print(score)

                    print(rank+1)
#     print('summary score for selected features: ',ranking_selected)
    for i,col in enumerate(final_columns):
        if rfecv.support_[i] == False:
            ranking_all_features[col] = rfecv.ranking_[i] 

    if len(ranking_all_features) != len(rfecv.support_):
        print("Difference in len between ranking and support")
    return ranking_all_features

def make_plot(rfecv, type_model):
    plt.figure(figsize=(16, 9))
    plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
    plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
    plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
    plt.plot(range(1, len(rfecv.grid_scores_) + 1), [score*-1 for score in rfecv.grid_scores_], color='#303F9F', linewidth=3)
    plt.savefig(f"results/{type_model}_overview_feature_selection.png")
    
data = Datahandling('scores_rfc_columns.csv')
highest_iter = data.highest_iteration
df_models_scores = data.current_df
df_scores = pd.DataFrame(index=final_columns)
if 'rfc' in settings['wrapper_tech'].keys():
    t1 = time.time()
    for type_model in settings['models'].keys():
        t2 = time.time()
        if 'svm' ==type_model:
            model = SVC(**settings['models']['svm'])
        elif 'rf' == type_model:
            model = RandomForestClassifier(**settings['models']['rf'])
        
        rfecv = RFECV(estimator=model,
                      scoring=settings["scoring_type"],
                      cv=settings['cross_val_times'],
                     verbose=2).fit(X, y)  
        
        ranking_all_features = calculate_ranking(rfecv,final_columns,type_model)
        scores = {column:{} for column in final_columns}
        for i in range(len(final_columns)):
            scores[final_columns[i]] = {
                f"selected_{type_model}":rfecv.support_[i],
                f"ranking_{type_model}":ranking_all_features[final_columns[i]]} #selected features are always 1 

        summary = [{
            'wrapper':type_model,
            'iteration':highest_iter +1,
            'grid_scores': rfecv.grid_scores_,
            'scores':scores,
            'optimal_number_of_features':rfecv.n_features_,
            'score_optimal_features':rfecv.grid_scores_[rfecv.n_features_-1],
            'scoring_type':settings['scoring_type'],
            "cross_val_times":2,
            "datapoints":settings['datapoints'],
            'params':settings['models'][type_model],
            'running_time':time.time() - t2,
            'scores':scores,
            'ranking':[ranking_all_features[col] for col in final_columns ]
        }
        ]
    

        df_models_scores = df_models_scores.append(summary,ignore_index=True)
        make_plot(rfecv, type_model)
#df_scores.to_csv("data/scores_rfc_columnsvvvvvvvv.csv")
data.save_after_iter(df_models_scores)
#df_models_scores.to_csv("data/scores_rfc_models.csv")
print(f"total running time: {time.time() - t1}")

    

In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
import time
import numpy as np
from sklearn.model_selection import cross_val_score
import winsound
from sklearn import preprocessing

In [2]:
settings = {
                "dataset":"data_after_cleaning_norm.csv",
                "columns_dataset": "scores_after_multicol.csv",
                "wrapper_tech": {
                        'rfc':{},
                        'ga':{},
                        'Boruta':{
                            "n_estimators":'auto',
                            "verbose":2
                            },
                        'xgboost':{}
                },
                "models":
                    {
                        'svm':
                             {
                                  'kernel':"linear"
                             },
                        'rf':
                             {
                                     'n_estimators':600,
                                     'max_depth':30,
                                     'min_samples_split':5,
                                     'min_samples_leaf':4,
                                     'max_features':'auto',
                                     'bootstrap':True,
                                     'n_jobs':-1
                             }
                    },
                "scoring_type":'accuracy',
                "cross_val_times":2,
                "datapoints":50000, #fill in all if you want to use all datapoints,
                "allow_creating_new_results_df":True, #only allow making new results df when True (risky to keep on true after testing)


}

In [3]:
data = pd.read_csv('data/' + settings["dataset"])

data2 = data
data2['difference'] = data2['close_price_next_min']-data2['ETHBTC__ticker_info__close_price']
data2['dummy_next_start_time'] = data2['difference'].apply(lambda x: 1 if x > 0 else 0) 
data2 = data2.drop(columns=['difference','close_price_next_min','last_start_time'])

columns = list(data2.columns)
columns.append("dummy_next_start_time") 
final_columns = [column for column in columns if column!="dummy_next_start_time"]

In [4]:
X = data2[final_columns].values
y = data2["dummy_next_start_time"].values

In [5]:
#reduce dataset if needed
if settings['datapoints'] is not 'all':
    X = X[:settings['datapoints']]
    y = y[:settings['datapoints']]
        

In [6]:
len(X),len(y)

(50000, 50000)

### SVC

In [ ]:
model_svc = SVC(kernel="linear")

rfecv_svc = RFECV(estimator=model_svc,
                      scoring=settings["scoring_type"],
                      min_features_to_select = 300,
                      cv = 3,
                      n_jobs = -1,
                      verbose=2).fit(X, y)


In [ ]:
rfecv_svc

In [ ]:
svc_rfe_scores = list(rfecv_svc.estimator_.coef_[0])


In [ ]:
kolommen = list(data2.columns)
kolommen = kolommen[:343]

In [ ]:
SVC_RFE_importance_ranking = pd.DataFrame()

SVC_RFE_importance_ranking['var'] = kolommen
SVC_RFE_importance_ranking['imp'] = svc_rfe_scores

In [ ]:
SVC_RFE_importance_ranking.head()

In [ ]:
SVC_RFE_importance_ranking = SVC_RFE_importance_ranking.sort_values(by=['imp'], ascending=False)


In [ ]:
SVC_RFE_importance_ranking = SVC_RFE_importance_ranking.reset_index(drop=True)


In [ ]:
# SVC_RFE_importance_ranking.to_csv(r'results/Feature_importance_ranking/SVC_RFE_Ranking2.csv', index = False)

### RF

In [ ]:
model_rfc = RandomForestClassifier(    n_estimators =600,
                                       max_depth=30,
                                       min_samples_split=5,
                                       min_samples_leaf=4,
                                       max_features='auto',
                                       bootstrap=True,
                                       n_jobs=-1)
    
rfecv = RFECV(estimator=model_rfc,
                      scoring=settings["scoring_type"],
                      min_features_to_select = 300,
                      verbose=2).fit(X, y)

In [ ]:
scores_345_rf_rfe = list(rfecv.estimator_.feature_importances_)


In [ ]:
kolommen = list(data2.columns)
kolommen = kolommen[:343]

In [ ]:
RF_RFE_importance_ranking = pd.DataFrame()


RF_RFE_importance_ranking['var'] = kolommen
RF_RFE_importance_ranking['imp'] = scores_345_rf_rfe

In [ ]:
RF_RFE_importance_ranking = RF_RFE_importance_ranking.sort_values(by=['imp'], ascending=False)


In [ ]:
RF_RFE_importance_ranking = RF_RFE_importance_ranking.reset_index(drop=True)
RF_RFE_importance_ranking.head()

In [ ]:
# RF_RFE_importance_ranking.to_csv(r'results/Feature_importance_ranking/RE_RFE_Ranking.csv', index = False)

<br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br><br>
<br>
<br>
<br>
<br>

In [ ]:
samen = {}
for i in range(len(kolommen)):
    samen[kolommen[i]] = scores_345_rf_rfe[i]

In [ ]:
import operator
samen_reversed = sorted(samen.items(),key=operator.itemgetter(1),reverse=True)

In [ ]:
samen_reversed

In [ ]:
# np.save('ranking_rf_rfe_345.npy', samen_reversed) 

In [ ]:
scores_345_rf_rfe.sort(reverse=True)
print(scores_345_rf_rfe)

In [ ]:
cumm = np.cumsum(scores_345_rf_rfe)

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt 

In [ ]:
import matplotlib.pyplot as plt

fig1 = plt.subplots()  
plt.plot(a)
plt.xlabel('number of features')
plt.ylabel('importance')
plt.show()

In [ ]:
plt.style.use('ggplot');
fig1 = plt.subplots()  
plt.plot(a)
plt.xlabel('number of features')
plt.ylabel('importance')
plt.show()

In [ ]:

plt.xlabel('number of features')
plt.ylabel('Importance')
plt.plot(x, rf_rfe_test['cum'], '-')
# plt.vlines(kn.knee, plt.ylim()[0], plt.ylim()[1], linestyles='dashed')
plt.show()